In [ ]:
import json
import random
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans
from tqdm import tqdm
from spacy.training import Example
import pandas as pd
from sklearn.metrics import classification_report
from spacy.training import offsets_to_biluo_tags
from spacy.tokens import Span
from spacy.training import Example, offsets_to_biluo_tags
from spacy.util import minibatch
from sklearn.metrics import precision_recall_fscore_support
import re
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from pathlib import Path


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import random
import spacy
import re
from spacy.training import Example, offsets_to_biluo_tags
from spacy.util import minibatch
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support


class SpacyNERTrainer:
    def __init__(self, data_path):
        self.data_path = data_path
        self.raw_data = self.load_jsonl_data()
        self.train_data, temp_data = train_test_split(self.raw_data, test_size=0.2, random_state=42)
        self.val_data, self.test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

        self.train_spacy = self.convert_to_spacy_format(self.train_data)
        self.val_spacy = self.convert_to_spacy_format(self.val_data)
        self.test_spacy = self.convert_to_spacy_format(self.test_data)

        self.nlp, self.ner = self.initialize_model()
        self.labels = self.add_labels()
        self.train_model()

    def load_jsonl_data(self):
        data = []
        with open(self.data_path, 'r', encoding='utf-8') as f:
            try:
                data = [json.loads(line) for line in f]
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line: {e}")
        return data

    def filter_valid_entities(self, text, entities):
        nlp = spacy.blank("en")
        doc = nlp.make_doc(text)
        valid_entities = []
        seen = set()

        entities.sort(key=lambda x: (x[0], - (x[1] - x[0])))  # sort by start, longest first
        last_end = -1

        for start, end, label in entities:
            if (start, end, label) in seen:
                print(f"Skipping duplicate entity: ({start}, {end}, {label})")
                continue
            if start < last_end:
                print(f"Skipping overlapping entity: ({start}, {end}, {label})")
                continue
            try:
                biluo = offsets_to_biluo_tags(doc, [(start, end, label)])
                if '-' not in biluo:
                    valid_entities.append((start, end, label))
                    last_end = end
                    seen.add((start, end, label))
                else:
                    print(f"Skipping misaligned entity: ({start}, {end}, {label})")
            except ValueError as e:
                print(f"Skipping entity due to alignment error: ({start}, {end}, {label}) - {e}")
        return valid_entities

    def convert_to_spacy_format(self, data):
        nlp = spacy.blank("en")
        spacy_data = []

        for item in data:
            text = item['content'].replace("\n", " ")
            entities = []

            for ann in item.get('annotation', []):
                labels = ann.get('label', [])
                if not labels or not isinstance(labels, list):
                    point_text = ann.get('points', [{}])[0].get('text', '').lower()
                    if 'b.b.m' in point_text or 'b.e' in point_text or 'mba' in point_text:
                        labels = ['Degree']
                    elif 'oracle' in point_text or 'microsoft' in point_text or 'accenture' in point_text:
                        labels = ['Companies worked at']
                    else:
                        labels = ['UNKNOWN']
                    print(f"Assigned label {labels} to annotation: {ann}")

                for point in ann.get('points', []):
                    try:
                        point_start = point['start']
                        point_end = point['end']
                        point_text = point.get('text', '')

                        lstrip_diff = len(point_text) - len(point_text.lstrip())
                        rstrip_diff = len(point_text) - len(point_text.rstrip())
                        if lstrip_diff:
                            point_start += lstrip_diff
                        if rstrip_diff:
                            point_end -= rstrip_diff

                        for label in labels:
                            entities.append((point_start, point_end + 1, label))
                    except (KeyError, TypeError) as e:
                        print(f"Skipping invalid point in annotation {ann}: {e}")
                        continue

            filtered_entities = self.filter_valid_entities(text, entities)
            if filtered_entities:
                spacy_data.append((text, {"entities": filtered_entities}))

        return spacy_data

    def initialize_model(self):
        nlp = spacy.blank("en")
        if "ner" not in nlp.pipe_names:
            ner = nlp.add_pipe("ner")
        else:
            ner = nlp.get_pipe("ner")
        return nlp, ner

    def add_labels(self):
        labels = set()
        for _, annotations in self.train_spacy:
            for _, _, label in annotations.get('entities', []):
                labels.add(label)
        for label in labels:
            self.ner.add_label(label)
        return labels

    def train_model(self, n_iter=50):
        optimizer = self.nlp.begin_training()
        other_pipes = [pipe for pipe in self.nlp.pipe_names if pipe != "ner"]

        with self.nlp.disable_pipes(*other_pipes):
            for itn in range(n_iter):
                random.shuffle(self.train_spacy)
                losses = {}
                batches = minibatch(self.train_spacy, size=8)

                for batch in batches:
                    for text, annotations in batch:
                        doc = self.nlp.make_doc(text)
                        try:
                            example = Example.from_dict(doc, annotations)
                            self.nlp.update([example], drop=0.5, sgd=optimizer, losses=losses)
                        except ValueError as e:
                            print(f"Skipping example due to error: {e}")
                            continue

                val_loss = self.evaluate_model(self.val_spacy)
                print(f"Iteration {itn + 1}, Losses: {losses}, Validation Loss: {val_loss}")

    def evaluate_model(self, data):
        losses = {}
        for text, annotations in data:
            doc = self.nlp.make_doc(text)
            try:
                example = Example.from_dict(doc, annotations)
                self.nlp.update([example], losses=losses)
            except ValueError as e:
                print(f"Skipping evaluation example due to error: {e}")
                continue
        return losses.get("ner", 0.0)


    def print_evaluation_metrics(self):
        metrics = self.calculate_metrics()

        # Print NER Level Metrics
        print("\n" + "="*50)
        print("NER LEVEL METRICS".center(50))
        print("="*50)
        if metrics['ner_level']:
            ner_data = []
            for label, scores in metrics['ner_level'].items():
                if label == "overall":
                    continue
                ner_data.append([
                    label,
                    f"{scores['precision']:.3f}",
                    f"{scores['recall']:.3f}",
                    f"{scores['f1']:.3f}",
                    scores['support']
                ])

            # Add overall row
            overall = metrics['ner_level'].get('overall', {})
            if overall:
                ner_data.append([
                    "OVERALL",
                    f"{overall['precision']:.3f}",
                    f"{overall['recall']:.3f}",
                    f"{overall['f1']:.3f}",
                    overall['support']
                ])

            # Print table
            from tabulate import tabulate
            print(tabulate(
                ner_data,
                headers=["Label", "Precision", "Recall", "F1-Score", "Support"],
                tablefmt="grid",
                floatfmt=".3f"
            ))
        else:
            print("No NER level metrics available")

        # Print Token Level Metrics
        print("\n" + "="*50)
        print("TOKEN LEVEL METRICS".center(50))
        print("="*50)
        if metrics['token_level']:
            token_data = []
            for label, scores in metrics['token_level'].items():
                if label == "overall":
                    continue
                token_data.append([
                    label,
                    f"{scores['precision']:.3f}",
                    f"{scores['recall']:.3f}",
                    f"{scores['f1_score']:.3f}",
                    scores['support']
                ])

            # Add overall row
            overall = metrics['token_level'].get('overall', {})
            if overall:
                token_data.append([
                    "OVERALL",
                    f"{overall['precision']:.3f}",
                    f"{overall['recall']:.3f}",
                    f"{overall['f1_score']:.3f}",
                    overall['support']
                ])

            # Print table
            print(tabulate(
                token_data,
                headers=["Label", "Precision", "Recall", "F1-Score", "Support"],
                tablefmt="grid",
                floatfmt=".3f"
            ))
        else:
            print("No token level metrics available")
    def calculate_metrics(self):
        ner_true, ner_pred = [], []
        token_true, token_pred = [], []

        for text, annotations in self.test_spacy:
            true_entities = annotations.get("entities", [])
            doc = self.nlp(text)
            pred_entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]

            ner_true.append(true_entities)
            ner_pred.append(pred_entities)

            example = Example.from_dict(self.nlp.make_doc(text), annotations)
            true_doc = example.reference
            true_token_labels = ["O"] * len(true_doc)
            for ent in true_entities:
                start_char, end_char, label = ent
                for token in true_doc:
                    if token.idx >= start_char and token.idx + len(token) <= end_char:
                        true_token_labels[token.i] = label

            pred_token_labels = ["O"] * len(doc)
            for ent in pred_entities:
                start_char, end_char, label = ent
                for token in doc:
                    if token.idx >= start_char and token.idx + len(token) <= end_char:
                        pred_token_labels[token.i] = label

            token_true.extend(true_token_labels)
            token_pred.extend(pred_token_labels)

        flat_true_labels = []
        flat_pred_labels = []
        for true_ents, pred_ents in zip(ner_true, ner_pred):
            true_set = set(true_ents)
            pred_set = set(pred_ents)

            for true_ent in true_ents:
                _, _, true_label = true_ent
                matched = False
                for pred_ent in pred_ents:
                    if pred_ent[:2] == true_ent[:2]:
                        flat_true_labels.append(true_label)
                        flat_pred_labels.append(pred_ent[2])
                        matched = True
                        break
                if not matched:
                    flat_true_labels.append(true_label)
                    flat_pred_labels.append("O")

            for pred_ent in pred_ents:
                if pred_ent not in true_set:
                    flat_true_labels.append("O")
                    flat_pred_labels.append(pred_ent[2])

        ner_labels = sorted(set(flat_true_labels + flat_pred_labels) - {"O"})
        ner_metrics = {}
        if flat_true_labels and flat_pred_labels:
            precision, recall, f1, support = precision_recall_fscore_support(
                flat_true_labels, flat_pred_labels, labels=ner_labels, zero_division=0
            )
            for label, p, r, f, s in zip(ner_labels, precision, recall, f1, support):
                ner_metrics[label] = {
                    "precision": p, "recall": r, "f1": f, "support": s
                }

            precision, recall, f1, _ = precision_recall_fscore_support(
                flat_true_labels, flat_pred_labels, average="weighted", zero_division=0
            )
            ner_metrics["overall"] = {
                "precision": precision, "recall": recall, "f1": f,
                "support": len(flat_true_labels)
            }

        token_labels = sorted(set(token_true + token_pred))
        token_metrics = {}
        if token_true and token_pred:
            precision, recall, f1, support = precision_recall_fscore_support(
                token_true, token_pred, labels=token_labels, zero_division=0
            )
            for label, p, r, f, s in zip(token_labels, precision, recall, f1, support):
                token_metrics[label] = {
                    "precision": p, "recall": r, "f1_score": f, "support": s
                }

            precision, recall, f1, _ = precision_recall_fscore_support(
                token_true, token_pred, average="weighted", zero_division=0
            )
            token_metrics["overall"] = {
                "precision": precision, "recall": recall, "f1_score": f,
                "support": len(token_true)
            }

        return {
            "ner_level": ner_metrics,
            "token_level": token_metrics
        }


# Usage
data_path = "/content/drive/MyDrive/Entity.json"
trainer = SpacyNERTrainer(data_path)
trainer.print_evaluation_metrics()


/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Sharan Adla - Email me on Indeed: indeed.com/r/Sha..." with entities "[(1801, 1842, 'Email Address')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Skipping misaligned entity: (1801, 1842, Email Address)


/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Akansha Jain Pune, Maharashtra - Email me on Indee..." with entities "[(1012, 1021, 'Companies worked at')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Akansha Jain Pune, Maharashtra - Email me on Indee..." with entities "[(1050, 1071, 'Degree')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Akansha Jain Pune, Mahar

Skipping misaligned entity: (1012, 1021, Companies worked at)
Skipping misaligned entity: (1050, 1071, Degree)
Skipping misaligned entity: (1073, 1095, College Name)
Skipping misaligned entity: (1131, 1135, Graduation Year)
Skipping misaligned entity: (1263, 1378, Skills)
Skipping misaligned entity: (1502, 1600, Skills)
Skipping misaligned entity: (1514, 1523, Companies worked at)
Skipping misaligned entity: (1573, 1577, Graduation Year)
Skipping misaligned entity: (1860, 1872, Name)


/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Rupesh Reddy Technology Consultant - EIT Services ..." with entities "[(3108, 3150, 'Email Address')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Rupesh Reddy Technology Consultant - EIT Services ..." with entities "[(3232, 3241, 'Location')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Rupesh Reddy Technology Cons

...

Skipping misaligned entity: (2643, 2676, Degree)
Skipping misaligned entity: (2678, 2719, College Name)
Skipping misaligned entity: (2721, 2725, UNKNOWN)
Skipping misaligned entity: (2912, 3288, Skills)


/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Zaheer Uddin Technical Project Manager  Hyderabad,..." with entities "[(1369, 1402, 'Designation')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Zaheer Uddin Technical Project Manager  Hyderabad,..." with entities "[(1404, 1415, 'Companies worked at')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Zaheer Uddin Techni

Skipping misaligned entity: (1369, 1402, Designation)
Skipping misaligned entity: (1404, 1415, Companies worked at)
Skipping misaligned entity: (4838, 4841, Degree)
Skipping misaligned entity: (4843, 4861, College Name)
Skipping misaligned entity: (4901, 4910, Location)


/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "VARUN AHLUWALIA Quantitative Analyst  - Email me o..." with entities "[(61, 105, 'Email Address')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Skipping misaligned entity: (61, 105, Email Address)
Skipping example due to error: [E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're training a named entity recognizer, also make sure that none of your annotated entity spans have leading or trailing whitespace or punctuation. You can also use the `debug data` command to validate your JSON-formatted training data. For details, run:
python -m spacy debug data --help
Skipping example due to error: [E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're training a named entity recognizer, also make sure that none of your annotated